# Regressão Linear, roda 30x, usa kfold para divisão. (itera entre os hiperparâmetros)

In [12]:
import os
import numpy as np
import pandas as pd

def load_txt_files_from_folders(root_directory):
    """
    Percorre todas as pastas dentro do diretório raiz e carrega arquivos .txt de cada pasta.

    Args:
        root_directory (str): Caminho para o diretório raiz.

    Returns:
        dict: Um dicionário com as informações carregadas.
              Chave = 'nome_da_pasta/nome_do_arquivo'
              Valor = (cabeçalho, np.array com os dados)
    """
    relevant_features_data = {}

    # Percorre todas as subpastas dentro do diretório raiz
    for folder_name in os.listdir(root_directory):
        folder_path = os.path.join(root_directory, folder_name)

        # Verifica se é uma pasta
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith('.txt'):
                    file_path = os.path.join(folder_path, filename)
                    try:
                        # Lendo os dados do arquivo
                        with open(file_path, 'r') as file:
                            lines = file.readlines()
                            header = lines[0].strip()  # Mantém o cabeçalho
                            data = [line.strip().split(',') for line in lines[1:] if line.strip()]  # Supondo CSV-like

                            # Adiciona ao dicionário com chave como 'pasta/arquivo'
                            key = f"{folder_name}/{filename}"
                            relevant_features_data[key] = (header, np.array(data, dtype=float))

                    except Exception as e:
                        print(f"Erro ao processar o arquivo {file_path}: {e}")
    
    return relevant_features_data


# Exemplo de uso:
root_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output3"
datasets = load_txt_files_from_folders(root_directory)

# Exibindo o resumo dos arquivos carregados
for key, (header, data) in datasets.items():
    print(f"Arquivo processado: {key}")
    print(f"Header: {header}")
    print(f"Shape dos dados: {data.shape}")
    print("-" * 50)


Arquivo processado: 1-saida/cocomo81.txt
Header: PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,PCA_11,PCA_12,actual
Shape dos dados: (63, 13)
--------------------------------------------------
Arquivo processado: 1-saida/encoded_china.txt
Header: PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,PCA_11,Effort
Shape dos dados: (499, 12)
--------------------------------------------------
Arquivo processado: 1-saida/encoded_desharnais.txt
Header: PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,PCA_11,PCA_12,PCA_13,PCA_14,PCA_15,Effort
Shape dos dados: (81, 16)
--------------------------------------------------
Arquivo processado: 1-saida/encoded_maxwell.txt
Header: PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,PCA_11,PCA_12,PCA_13,PCA_14,PCA_15,PCA_16,PCA_17,PCA_18,PCA_19,PCA_20,PCA_21,effort_D
Shape dos dados: (63, 22)
--------------------------------------------------
Arquivo processado: 10-saida/cocomo81.txt
Header: PCA_1,PCA_2,PCA_

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import (
    mean_squared_error,
    r2_score,
)
import numpy as np
import pandas as pd

# Configurar KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # 10 divisões para validação cruzada

# Resultados agregados
metrics_summary = []
best_hyperparameters = []  # Lista para armazenar os melhores hiperparâmetros por dataset

# Hiperparâmetros possíveis para LinearRegression
hyperparameters = {
    "fit_intercept": [True, False],
    "positive": [True, False],
}

for filename, (header, data) in datasets.items():
    # Separar as variáveis X e y
    X = data[:, :-1]
    y = data[:, -1]

    # Inicializar listas para as métricas
    mse_list, r2_list = [], []
    hyperparameter_results = []  # Armazena métricas para cada combinação de hiperparâmetros

    for _ in range(30):
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # Iterar sobre combinações de hiperparâmetros
            for fit_intercept in hyperparameters["fit_intercept"]:
                for positive in hyperparameters["positive"]:
                    try:
                        # Modelo de Regressão Linear com hiperparâmetros
                        model = LinearRegression(
                            fit_intercept=fit_intercept,
                            positive=positive,
                        )
                        # Ajustar o modelo
                        model.fit(X_train, y_train)
                        y_pred = model.predict(X_test)

                        # Calcular as métricas e armazenar
                        mse = mean_squared_error(y_test, y_pred)
                        r2 = r2_score(y_test, y_pred)
                        mse_list.append(mse)
                        r2_list.append(r2)

                        # Armazenar os resultados dos hiperparâmetros
                        hyperparameter_results.append({
                            "fit_intercept": fit_intercept,
                            "positive": positive,
                            "MSE": mse,
                            "R²": r2,
                        })
                    except RuntimeError as e:
                        print(f"Erro de convergência no arquivo {filename} com fit_intercept={fit_intercept}, positive={positive}: {e}")
                    except Exception as e:
                        print(f"Erro no arquivo {filename} com fit_intercept={fit_intercept}, positive={positive}: {e}")

    # Calcular as médias das métricas e encontrar os melhores hiperparâmetros
    if hyperparameter_results:  # Verifica se existem métricas válidas
        best_result = max(hyperparameter_results, key=lambda x: x["R²"])  # Seleciona o melhor R²
        metrics_summary.append({
            "Arquivo": filename,
            "MSE (Média)": best_result["MSE"],
            "R² (Média)": best_result["R²"],
        })
        best_hyperparameters.append({
            "Arquivo": filename,
            "Melhor Fit Intercept": best_result["fit_intercept"],
            "Melhor Positive": best_result["positive"],
        })

# Converter os resultados para DataFrames
metrics_df = pd.DataFrame(metrics_summary)
hyperparameters_df = pd.DataFrame(best_hyperparameters)

# Salvar os resultados em arquivos Excel
metrics_df.to_excel("metrics_regressao_linear_saida_test.xlsx", index=False)
hyperparameters_df.to_excel("best_hyperparameters_saida_4.xlsx", index=False)

# Exibir os resultados
print("Resultados salvos em metrics_regressao_linear_saida_4.xlsx e best_hyperparameters_saida_4.xlsx")


Resultados salvos em metrics_regressao_linear_saida_4.xlsx e best_hyperparameters_saida_4.xlsx
